In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [2]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],  # стаж
              [500, 700, 750, 600, 1450,        # средняя стоимость занятия
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 1, 3, 3, 1, 2]], dtype = np.float64) # квалификация репетитора

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1]) # подходит или нет репетитор

In [3]:
def calc_std_feat(x):
    res = (x - x.mean()) / x.std()
    return res

X_st = X.copy()
X_st[2, :] = calc_std_feat(X[2, :])
X_st[2, :]

array([-0.97958969, -0.56713087, -0.46401617, -0.77336028,  0.97958969,
       -0.36090146,  1.08270439,  2.11385144, -1.08270439,  0.05155735])

In [4]:
def calc_logloss(y, y_pred):
    err = np.mean(- y * np.log(y_pred) - (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [5]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [57]:
def eval_LR_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    w = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations + 1):
        
        
        z = np.dot(w, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        
        #y_pred = np.dot(w, X)
        #err = calc_mse(y, y_pred)

        w -= alpha * (1/n * np.dot((y_pred - y), X.T))
        if i % (iterations / 10) == 0:
            print(i, w, err)
    return w

### Task 1 - змените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log (как вариант - np.clip

In [44]:
def calc_logloss(y, y_pred):
    y_pred_cor = np.clip(y_pred, 1e-12,a_max=None)
    err = np.mean(- y * np.log(y_pred_cor) - (1.0 - y) * np.log(1.0 - y_pred_cor))
    return err

In [45]:
W = eval_LR_model(X_st, y, iterations=1000, alpha=1e-4)

 Iteration 100, Weights [ 0.49282748 -0.15007528  0.64748973  1.51727915], Proba [0.7731123  0.81653305 0.9491737  0.79567858 0.97615259 0.85526688
 0.99328138 0.9927105  0.76118791 0.96306664]
 Iteration 200, Weights [ 0.48896219 -0.16184918  0.64728128  1.51155738], Proba [0.7693796  0.81332918 0.94727084 0.79221028 0.9749401  0.8540854
 0.99272495 0.99163077 0.75732455 0.96165894]
 Iteration 300, Weights [ 0.48511874 -0.17358386  0.64706349  1.50586552], Proba [0.76562206 0.81009717 0.94530934 0.7887151  0.97367248 0.85290285
 0.99212512 0.99039726 0.75343772 0.9602059 ]
 Iteration 400, Weights [ 0.4812976  -0.18527698  0.64683669  1.50020462], Proba [0.76184106 0.80683807 0.94328841 0.78519423 0.97234795 0.85171952
 0.99147891 0.98898954 0.74952885 0.95870677]
 Iteration 500, Weights [ 0.47749927 -0.19692597  0.64660127  1.4945758 ], Proba [0.75803804 0.803553   0.94120737 0.78164897 0.97096471 0.8505357
 0.99078317 0.98738483 0.74559949 0.95716084]
 Iteration 600, Weights [ 0.4737

### Task 2 - Подберите аргументы функции eval_LR_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [7]:
def eval_LR_model(X, y, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    i = 0
    delta = 1e-5
    err = np.inf
    
    while True:
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err_new = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
            
        if abs(err - err_new) <= delta:
            return(i, W, err)
        else:
            err = err_new
            i += 1
    return W

In [8]:
for alpha in np.logspace(-4,-1,10):
    i, W, err = eval_LR_model(X_st, y, alpha)
    print(f'Iteration {i}, Weights {W}, error {err}')

Iteration 6889, Weights [ 0.30484831 -0.64299824  0.65968825  1.27144238], error 0.608860244923452
Iteration 4152, Weights [ 0.27267716 -0.6733186   0.67829466  1.24905159], error 0.594972380633772
Iteration 3094, Weights [ 0.20548233 -0.69490797  0.73001333  1.22223523], error 0.5789668242611163
Iteration 3016, Weights [ 0.06146141 -0.68603187  0.85260956  1.21305757], error 0.5558287097554773
Iteration 2565, Weights [-0.09924194 -0.68920757  0.96824219  1.25685066], error 0.5390646883831742
Iteration 3872, Weights [-0.57606471 -0.79958528  1.11265147  1.66333134], error 0.5036484978805061
Iteration 6822, Weights [-2.01991797 -0.97320565  0.83993406  2.78408516], error 0.43337113257078075
Iteration 7570, Weights [-3.99609587 -1.02006284  0.08932723  4.05101538], error 0.36853676125653717
Iteration 6593, Weights [-5.97956702 -1.091073   -0.64593205  5.41143908], error 0.32369553197603584
Iteration 5852, Weights [-8.53579974 -1.24563049 -1.51151549  7.33020294], error 0.2838539721501477

### Task 3 - Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются веса, которые уже посчитаны функцией eval_LR_model и X, на выходе - массив y_pred_proba)

In [9]:
def calc_pred_proba(W, X):
    y_pred_proba = sigmoid(np.dot(W, X))
    return y_pred_proba

In [10]:
def eval_LR_model_proba(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    w = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations + 1):
        y_pred = calc_pred_proba(w, X)
        err = calc_logloss(y, y_pred)
        
        #y_pred = np.dot(w, X)
        #err = calc_mse(y, y_pred)

        w -= alpha * (1/n * np.dot((y_pred - y), X.T))
        if i % (iterations / 10) == 0:
            print(f' Iteration {i}, Weights {w}, Proba {y_pred}')
   

In [11]:
eval_LR_model_proba(X_st, y, iterations=1000, alpha=1e-2)

 Iteration 100, Weights [ 0.25769171 -0.68297243  0.68841418  1.2405903 ], Proba [0.53607431 0.60541814 0.7422915  0.57109511 0.79705706 0.77767728
 0.78843285 0.19985227 0.51840725 0.80413529]
 Iteration 200, Weights [ 0.14805533 -0.69317972  0.77965488  1.21287192], Proba [0.47645678 0.55650611 0.6957196  0.51658785 0.77869268 0.74665932
 0.76220483 0.18266285 0.45647252 0.77356346]
 Iteration 300, Weights [ 0.06257746 -0.68605867  0.85181735  1.21283095], Proba [0.43935049 0.52679455 0.67298372 0.48294194 0.7797799  0.72490536
 0.76722525 0.2042211  0.41785907 0.76143456]
 Iteration 400, Weights [-0.00887935 -0.68415387  0.90713736  1.22443854], Proba [0.41187041 0.50442949 0.65731176 0.45778572 0.78177463 0.70865186
 0.77303149 0.22091504 0.38942549 0.75376301]
 Iteration 500, Weights [-0.0701621  -0.68661758  0.94984062  1.24404686], Proba [0.39116152 0.48729838 0.64664623 0.4386544  0.78387724 0.69668605
 0.77850519 0.23203289 0.36810938 0.74910571]
 Iteration 600, Weights [-0.12

### Task 4 - Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются веса, которые уже посчитаны функцией eval_LR_model и X, на выходе - массив y_pred).

In [12]:
def calc_pred(W, X):
    y_pred_tmp = sigmoid(np.dot(W, X))
    y_pred = np.zeros(y_pred_tmp.shape[0]) 

    for i in range(y_pred_tmp.shape[0]):
        if (y_pred_tmp[i] > 0.5): 
            y_pred[i] = 1
        else:
            y_pred[i] = 0
    
    return y_pred

In [13]:
calc_pred(W, X_st)

array([0., 0., 1., 0., 1., 0., 1., 0., 0., 1.])

### Task 7 - Создайте функции eval_LR_model_l1 и eval_LR_model_l2 с применением L1 и L2 регуляризации соответственно.

In [14]:
def eval_model_l1(X, y, iterations, alpha=1e-4, lambda_=1e-8):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        y_pred = sigmoid(np.dot(W, X))
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T) + np.sign(W) * lambda_/2)
        if i % (iterations / 10) == 0:
            print(f'Iteration {i}, Weights {W}, Error {err}')
    return W

In [15]:
def eval_model_l2(X, y, iterations, alpha=1e-4, lambda_=1e-8):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        y_pred = sigmoid(np.dot(W, X))
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T) + lambda_ * W)
        if i % (iterations / 10) == 0:
            print(f'Iteration {i}, Weights {W}, Error {err}')
    return W

In [16]:
eval_model_l1(X_st, y, iterations=10000, alpha=1e-8)

Iteration 1000, Weights [ 0.49671026 -0.13827613  0.64768834  1.52302409], Error 1.2200468091973615
Iteration 2000, Weights [ 0.49670636 -0.13828796  0.64768815  1.52301833], Error 1.2200279718986276
Iteration 3000, Weights [ 0.49670246 -0.13829979  0.64768796  1.52301256], Error 1.2200091347312805
Iteration 4000, Weights [ 0.49669856 -0.13831162  0.64768776  1.5230068 ], Error 1.2199902976953312
Iteration 5000, Weights [ 0.49669467 -0.13832344  0.64768757  1.52300103], Error 1.219971460790777
Iteration 6000, Weights [ 0.49669077 -0.13833527  0.64768737  1.52299527], Error 1.219952624017627
Iteration 7000, Weights [ 0.49668687 -0.1383471   0.64768718  1.5229895 ], Error 1.2199337873758913
Iteration 8000, Weights [ 0.49668298 -0.13835893  0.64768699  1.52298374], Error 1.2199149508655678
Iteration 9000, Weights [ 0.49667908 -0.13837076  0.64768679  1.52297798], Error 1.2198961144866662
Iteration 10000, Weights [ 0.49667518 -0.13838259  0.6476866   1.52297221], Error 1.2198772782391871


array([ 0.49667518, -0.13838259,  0.6476866 ,  1.52297221])

In [17]:
eval_model_l2(X_st, y, iterations=10000, alpha=1e-8)

Iteration 1000, Weights [ 0.49671026 -0.13827613  0.64768834  1.52302409], Error 1.2200468091972592
Iteration 2000, Weights [ 0.49670636 -0.13828796  0.64768815  1.52301833], Error 1.2200279718984213
Iteration 3000, Weights [ 0.49670246 -0.13829979  0.64768796  1.52301256], Error 1.220009134730977
Iteration 4000, Weights [ 0.49669856 -0.13831162  0.64768776  1.5230068 ], Error 1.219990297694922
Iteration 5000, Weights [ 0.49669467 -0.13832344  0.64768757  1.52300103], Error 1.2199714607902672
Iteration 6000, Weights [ 0.49669077 -0.13833527  0.64768737  1.52299527], Error 1.219952624017019
Iteration 7000, Weights [ 0.49668687 -0.1383471   0.64768718  1.5229895 ], Error 1.219933787375179
Iteration 8000, Weights [ 0.49668298 -0.13835893  0.64768699  1.52298374], Error 1.219914950864753
Iteration 9000, Weights [ 0.49667908 -0.13837076  0.64768679  1.52297798], Error 1.2198961144857479
Iteration 10000, Weights [ 0.49667518 -0.13838259  0.6476866   1.52297221], Error 1.2198772782381664


array([ 0.49667518, -0.13838259,  0.6476866 ,  1.52297221])